In [1]:
import anndata
import numpy as np
import sys
import torch
sys.path.append('../../../')
import velovae as vv

In [2]:
dataset = 'Pancreas'
adata = anndata.read_h5ad(f'data/{dataset}_pp.h5ad')

In [ ]:
#uncomment the next line if data is not preprocessed
#vv.preprocess(adata, n_gene=2000, min_shared_counts=20)

In [3]:
model_path_base = f'checkpoints/{dataset}'
figure_path_base = f'figures/{dataset}'
data_path = f'data/velovae/discrete/{dataset}'
gene_plot = ['Cpe','Gng12', 'Ppp3ca', 'Smoc1']

# Discrete VAE

In [4]:
figure_path = f'{figure_path_base}/DVAE'
model_path = f'{model_path_base}/DVAE'

torch.manual_seed(2022)
np.random.seed(2022)

dvae = vv.model.DVAE(adata, 
                     tmax=20, 
                     dim_z=5,
                     init_method='random',
                     device='cuda:0')

dvae.train(adata, plot=False, gene_plot=gene_plot, figure_path=figure_path)

dvae.save_model(model_path, 'encoder', 'decoder')
dvae.save_anndata(adata, 'dvae', data_path, f'{dataset}.h5ad')

Detecting zero scaling factors: 0, 0
Random Initialization.
Gaussian Prior.
Library scale (U): Max=4.93, Min=0.20, Mean=0.98
Library scale (S): Max=2.78, Min=0.44, Mean=1.02
--------------------------- Train a VeloVAE ---------------------------
*********        Creating Training/Validation Datasets        *********
*********                      Finished.                      *********
*********                 Creating optimizers                 *********
*********                      Finished.                      *********
*********                    Start training                   *********
*********                      Stage  1                       *********
Total Number of Iterations Per Epoch: 21, test iteration: 40
*********       Stage 1: Early Stop Triggered at epoch 595.       *********
*********                      Stage  2                       *********


  0%|          | 0/3696 [00:00<?, ?it/s]

Percentage of Invalid Sets: 0.007
Average Set Size: 103
*********       Stage 2: Early Stop Triggered at epoch 843.       *********
*********              Finished. Total Time =   0 h :  3 m : 55 s             *********
Final: Train ELBO = -2280.876,           Test ELBO = -2367.377
       Training MSE = 18.314, Test MSE = 17.543


# Discrete Full VB

In [5]:
figure_path = f'{figure_path_base}/DFullVB'
model_path = f'{model_path_base}/DFullVB'

torch.manual_seed(2022)
np.random.seed(2022)

dvae = vv.model.DVAEFullVB(adata, 
                           tmax=20, 
                           dim_z=5, 
                           init_method='random',
                           device='cuda:0')

dvae.train(adata, plot=False, gene_plot=gene_plot, figure_path=figure_path, embed='umap')

dvae.save_model(model_path, 'encoder', 'decoder')
dvae.save_anndata(adata, 'dfullvb', data_path, f'{dataset}.h5ad')

Detecting zero scaling factors: 0, 0
Random Initialization.
Gaussian Prior.
Library scale (U): Max=4.93, Min=0.20, Mean=0.98
Library scale (S): Max=2.78, Min=0.44, Mean=1.02
--------------------------- Train a VeloVAE ---------------------------
*********        Creating Training/Validation Datasets        *********
*********                      Finished.                      *********
*********                 Creating optimizers                 *********
*********                      Finished.                      *********
*********                    Start training                   *********
*********                      Stage  1                       *********
Total Number of Iterations Per Epoch: 21, test iteration: 40
*********       Stage 1: Early Stop Triggered at epoch 650.       *********
*********                      Stage  2                       *********


  0%|          | 0/3696 [00:00<?, ?it/s]

Percentage of Invalid Sets: 0.005
Average Set Size: 113
*********       Stage 2: Early Stop Triggered at epoch 1030.       *********
*********              Finished. Total Time =   0 h :  5 m : 30 s             *********
Final: Train ELBO = -2221.700,           Test ELBO = -2337.895
       Training MSE = 11.245, Test MSE = 13.241


# Evaluation

In [ ]:
cluster_edges = [('Ngn3 high EP', 'Pre-endocrine'),
                 ('Pre-endocrine', 'Delta'), 
                 ('Pre-endocrine', 'Beta'), 
                 ('Pre-endocrine','Epsilon'), 
                 ('Pre-endocrine','Alpha')]
vv.post_analysis(adata,
                 'eval',
                 ['Discrete VeloVAE', 'Discrete FullVB'],
                 ['dvae', 'dfullvb'],
                 compute_metrics=True,
                 genes=gene_plot,
                 grid_size=(1,4),
                 plot_type=['all'],
                 save_path=data_path,
                 cluster_edges=cluster_edges)